<a href="https://colab.research.google.com/github/SergioCarmo-ro/TelecomX_BR_Parte_II/blob/main/TelecomX_BR_Parte_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🎯 Missão

A nova missão é desenvolver modelos preditivos capazes de prever quais clientes têm maior chance de cancelar seus serviços.

A empresa quer antecipar o problema da evasão, e cabe a você construir um pipeline robusto para essa etapa inicial de modelagem.

📝 Extração do Arquivo Tratado

  Fazendo Leitura dos Dados

In [ ]:
import pandas as pd

In [ ]:
dados = pd.read_csv('/content/dados_tratados.csv')

In [ ]:
dados

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
...,...,...,...,...,...,...
7262,9987-LUTYD,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
7263,9992-RRAMN,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7264,9992-UJOEL,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7265,9993-LHIEB,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Two year', 'PaperlessBilling': '..."


In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7043 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


Eliminando Colunas irrelevantes

In [ ]:
# Colunas que são identificadores únicos ou não contribuem para a previsão
colunas_irrelevantes = ['ID', 'ClienteID', 'Nome', 'CPF', 'Telefone', 'Email']  # Exemplo de colunas a remover
colunas_a_remover = [col for col in colunas_irrelevantes if col in dados.columns]  # Verifica se as colunas existem
dados = dados.drop(columns=colunas_a_remover)

In [ ]:
# Colunas removidas e  que permaneceram
print("Colunas removidas:", colunas_a_remover)
print("Colunas restantes:", dados.columns.tolist())

Colunas removidas: []
Colunas restantes: ['Churn', 'customer', 'phone', 'internet', 'account']


 Identificando variáveis categóricas

In [ ]:
# Vamos assumir que variáveis categóricas são do tipo 'object' ou 'category'
colunas_categoricas = dados.select_dtypes(include=['object', 'category']).columns.tolist()
print("Variáveis categóricas detectadas:", colunas_categoricas)

Variáveis categóricas detectadas: ['Churn', 'customer', 'phone', 'internet', 'account']


Aplicando one-hot encoding nas variáveis categóricas

In [ ]:
# Criar o transformador de colunas
column_transformer = make_column_transformer(
    (OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), colunas_categoricas),
    remainder='passthrough'  # Mantém colunas numéricas sem alteração
)

Aplicando a transformação

In [ ]:
# Aplicar a transformação
dados_transformados = column_transformer.fit_transform(dados)

In [ ]:
# Obter nomes das novas colunas após one-hot encoding
novas_colunas = column_transformer.get_feature_names_out()

In [ ]:
# Criar DataFrame com os dados transformados
dados_transformados = pd.DataFrame(dados_transformados, columns=novas_colunas)

In [ ]:
# Exibindo as primeiras linhas do DataFrame codificado
print("\nPrimeiras 5 linhas do dataset codificado:")
display(dados_transformados.head())


Primeiras 5 linhas do dataset codificado:


,onehotencoder__Churn_Yes,onehotencoder__Churn_nan,"onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 12}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 14}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 15}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 16}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 17}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 18}",...,"onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 85.25, 'Total': '5714.2'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.25, 'Total': '4652.4'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.4, 'Total': '6376.55'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.45, 'Total': '5957.9'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.5, 'Total': '4318.35'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.6, 'Total': '5817.45'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 91.25, 'Total': '6589.6'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.15, 'Total': '6480.9'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.45, 'Total': '6460.55'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 95.5, 'Total': '4627.85'}}"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Salvar o resultado (opcional)
dados_transformados.to_csv('/content/dados_transformados.csv', index=False)

Separando a Variável Alvo das Variáveis Explicativas

In [ ]:
# Separando recursos e alvo

x = dados_transformados.copy()

y = dados['Churn']

# Verificação
if 'Churn' in x.columns:
    x = x.drop('Churn', axis=1)

display(x.head())
display(y.head())

,onehotencoder__Churn_Yes,onehotencoder__Churn_nan,"onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 12}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 14}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 15}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 16}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 17}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 18}",...,"onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 85.25, 'Total': '5714.2'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.25, 'Total': '4652.4'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.4, 'Total': '6376.55'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.45, 'Total': '5957.9'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.5, 'Total': '4318.35'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.6, 'Total': '5817.45'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 91.25, 'Total': '6589.6'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.15, 'Total': '6480.9'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.45, 'Total': '6460.55'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 95.5, 'Total': '4627.85'}}"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Churn
0,No
1,No
2,Yes
3,Yes
4,Yes


Verificação da Proporção de Evasão:

In [ ]:
# Verificar os valores únicos na coluna 'Churn'
print("Valores únicos na coluna 'Churn':", dados['Churn'].unique())

Valores únicos na coluna 'Churn': ['No' 'Yes' nan]


In [ ]:
# Calculando número de clientes que evadiram e que permaneceram
total_clientes = len(dados)
clientes_churn = len(dados[dados['Churn'] == 'Yes'])  # Assumindo que 'Yes' indica clientes que evadiram
clientes_ativos = len(dados[dados['Churn'] == 'No'])  # Assumindo que 'No' indica permaneceram ativos

In [ ]:
# Calculo das proporções
proporcao_churn = clientes_churn / total_clientes # indica clientes que evadiram
proporcao_ativos = clientes_ativos / total_clientes # indica clientes que permaneceram ativos

In [ ]:
# Calculo da proporção de evasão de clientes em relação aos ativos
proporcao_churn_vs_ativos = clientes_churn / clientes_ativos if clientes_ativos > 0 else 0

In [ ]:
# Exibindo resultados
print(f"Total de clientes: {total_clientes}")
print(f"Clientes que evadiram: {clientes_churn} ({proporcao_churn:.2%})")
print(f"Clientes ativos: {clientes_ativos} ({proporcao_ativos:.2%})")
print(f"Proporção de clientes que evadiram em relação aos ativos: {proporcao_churn_vs_ativos:.2f}")

Total de clientes: 7267
Clientes que evadiram: 1869 (25.72%)
Clientes ativos: 5174 (71.20%)
Proporção de clientes que evadiram em relação aos ativos: 0.36


Avaliando desequilíbrio entre as classe

In [ ]:
# Contar o número de clientes por classe
contagem_classes = dados['Churn'].value_counts()
total_clientes = len(dados)

In [ ]:
# Calcular proporções
proporcoes = contagem_classes / total_clientes

In [ ]:
# Exibir resultados
print("\nDistribuição das classes:")
for classe, contagem in contagem_classes.items():
    print(f"Classe {classe}: {contagem} ({proporcoes[classe]:.2%})")


Distribuição das classes:
Classe No: 5174 (71.20%)
Classe Yes: 1869 (25.72%)


In [ ]:
# Avaliar desequilíbrio
razao = contagem_classes.min() / contagem_classes.max()
print(f"\nRazão entre a classe minoritária e majoritária: {razao:.2f}")
if razao < 0.3:
    print("Aviso: Há um desequilíbrio significativo entre as classes (razão < 0.3).")
elif razao < 0.5:
    print("Nota: Há um desequilíbrio moderado entre as classes (razão < 0.5).")
else:
    print("As classes estão relativamente equilibradas.")


Razão entre a classe minoritária e majoritária: 0.36
Nota: Há um desequilíbrio moderado entre as classes (razão < 0.5).


Verificar se há a necessidade de normalizar ou padronizar os dados

In [ ]:
# Identificar colunas categóricas
colunas_categoricas = x.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
# Identificar colunas numéricas
colunas_numericas = x.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
# Exibir estatísticas descritivas das colunas numéricas
print("Estatísticas descritivas das variáveis numéricas (antes da transformação):")
print(x[colunas_numericas].describe())
display(x.head())



Estatísticas descritivas das variáveis numéricas (antes da transformação):
       onehotencoder__Churn_Yes  onehotencoder__Churn_nan  \
count               7267.000000               7267.000000   
mean                   0.257190                  0.030824   
std                    0.437115                  0.172853   
min                    0.000000                  0.000000   
25%                    0.000000                  0.000000   
50%                    0.000000                  0.000000   
75%                    1.000000                  0.000000   
max                    1.000000                  1.000000   

       onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}  \
count                                        7267.000000                                                                     
mean                                            0.003578                                                                   

,onehotencoder__Churn_Yes,onehotencoder__Churn_nan,"onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 12}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 14}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 15}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 16}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 17}","onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 18}",...,"onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 85.25, 'Total': '5714.2'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.25, 'Total': '4652.4'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.4, 'Total': '6376.55'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.45, 'Total': '5957.9'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.5, 'Total': '4318.35'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.6, 'Total': '5817.45'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 91.25, 'Total': '6589.6'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.15, 'Total': '6480.9'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.45, 'Total': '6460.55'}}","onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 95.5, 'Total': '4627.85'}}"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Verificando a escala das variáveis numéricas
if colunas_numericas:
    print("\nAnálise de escala das variáveis numéricas:")
    for col in colunas_numericas:
        minimo = x[col].min()
        maximo = x[col].max()
        amplitude = maximo - minimo
        print(f"{col}: Mínimo = {minimo:.2f}, Máximo = {maximo:.2f}, Amplitude = {amplitude:.2f}")

A saída de streaming foi truncada nas últimas 5000 linhas.
onehotencoder__account_{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Credit card (automatic)', 'Charges': {'Monthly': 90.85, 'Total': '4515.85'}}: Mínimo = 0.00, Máximo = 1.00, Amplitude = 1.00
onehotencoder__account_{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Credit card (automatic)', 'Charges': {'Monthly': 90.85, 'Total': '90.85'}}: Mínimo = 0.00, Máximo = 1.00, Amplitude = 1.00
onehotencoder__account_{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Credit card (automatic)', 'Charges': {'Monthly': 90.95, 'Total': '1612.2'}}: Mínimo = 0.00, Máximo = 1.00, Amplitude = 1.00
onehotencoder__account_{'Contract': 'Month-to-month', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Credit card (automatic)', 'Charges': {'Monthly': 91.05, 'Total': '4370.75'}}: Mínimo = 0.00, Máximo = 1.00, Amplitude = 1.00
onehotencoder__account_{'Contract': 'Month-to-mo

In [ ]:
# Verificando se há diferenças significativas nas escalas
if colunas_numericas:
    amplitude = [x[col].max() - x[col].min() for col in colunas_numericas]
    if len(amplitude) > 0:
        max_amplitude = max(amplitude)
        min_amplitude = min(amplitude)
        razao_amplitude = max_amplitude / min_amplitude if min_amplitude > 0 else float('inf')
        print(f"\nRazão entre maior e menor amplitude: {razao_amplitude:.2f}")
        if razao_amplitude > 10:
            print("Aviso: As variáveis têm escalas muito diferentes. Normalização ou padronização é recomendada.")
        else:
            print("As escalas das variáveis são relativamente próximas. Normalização ou padronização pode não ser estritamente necessária.")
else:
    print("\nNenhuma variável numérica encontrada no dataset original.")


Razão entre maior e menor amplitude: 1.00
As escalas das variáveis são relativamente próximas. Normalização ou padronização pode não ser estritamente necessária.


In [ ]:
# Criando o transformador de colunas (para categóricas)
column_transformer = make_column_transformer(
    (OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), colunas_categoricas),
    remainder='passthrough'
)

In [ ]:
# Aplicando a transformação
x_transformado = column_transformer.fit_transform(x)
novas_colunas = column_transformer.get_feature_names_out()
x_transformado = pd.DataFrame(x_transformado, columns=novas_colunas)

In [ ]:
# Identificando colunas numéricas após transformação (exclui colunas categóricas codificadas)
colunas_numericas_transformadas = [col for col in novas_colunas if 'remainder__' in col]

In [ ]:
# Aplicando normalização e padronização (exemplo)
from sklearn.preprocessing import MinMaxScaler, StandardScaler

if colunas_numericas_transformadas:
    # Normalização (MinMaxScaler)
    normalizer = make_column_transformer(
        (MinMaxScaler(), colunas_numericas_transformadas),
        remainder='passthrough'
    )
    x_normalizado = normalizer.fit_transform(x_transformado)
    x_normalizado = pd.DataFrame(x_normalizado, columns=novas_colunas)

    # Padronização (StandardScaler)
    standardizer = make_column_transformer(
        (StandardScaler(), colunas_numericas_transformadas),
        remainder='passthrough'
    )
    x_padronizado = standardizer.fit_transform(x_transformado)
    x_padronizado = pd.DataFrame(x_padronizado, columns=novas_colunas)

    # Salvar os datasets processados
    x_normalizado['Churn'] = y.reset_index(drop=True)
    x_padronizado['Churn'] = y.reset_index(drop=True)
    x_normalizado.to_csv('/content/dados_normalizados.csv', index=False)
    x_padronizado.to_csv('/content/dados_padronizados.csv', index=False)
    print("\nDatasets salvos: 'dados_normalizados.csv' e 'dados_padronizados.csv'.")
else:
    print("\nNenhuma variável numérica para normalizar/padronizar após a transformação.")


Datasets salvos: 'dados_normalizados.csv' e 'dados_padronizados.csv'.


In [ ]:
# Exibir estatísticas após normalização
print("\nEstatísticas após normalização (MinMaxScaler):")
print(x_normalizado[colunas_numericas_transformadas].describe())
display(x_normalizado.head())


Estatísticas após normalização (MinMaxScaler):
       remainder__onehotencoder__Churn_Yes  \
count                          7267.000000   
mean                              0.257190   
std                               0.437115   
min                               0.000000   
25%                               0.000000   
50%                               0.000000   
75%                               1.000000   
max                               1.000000   

       remainder__onehotencoder__Churn_nan  \
count                          7267.000000   
mean                              0.030824   
std                               0.172853   
min                               0.000000   
25%                               0.000000   
50%                               0.000000   
75%                               0.000000   
max                               1.000000   

       remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 't

,remainder__onehotencoder__Churn_Yes,remainder__onehotencoder__Churn_nan,"remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 12}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 14}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 15}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 16}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 17}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 18}",...,"remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.25, 'Total': '4652.4'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.4, 'Total': '6376.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.45, 'Total': '5957.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.5, 'Total': '4318.35'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.6, 'Total': '5817.45'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 91.25, 'Total': '6589.6'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.15, 'Total': '6480.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.45, 'Total': '6460.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 95.5, 'Total': '4627.85'}}",Churn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes


In [ ]:
# Exibir estatísticas após padronização
print("\nEstatísticas após padronização (StandardScaler):")
print(x_padronizado[colunas_numericas_transformadas].describe())
display(x_padronizado.head())


Estatísticas após padronização (StandardScaler):
       remainder__onehotencoder__Churn_Yes  \
count                         7.267000e+03   
mean                          3.715512e-17   
std                           1.000069e+00   
min                          -5.884211e-01   
25%                          -5.884211e-01   
50%                          -5.884211e-01   
75%                           1.699463e+00   
max                           1.699463e+00   

       remainder__onehotencoder__Churn_nan  \
count                         7.267000e+03   
mean                         -4.595501e-17   
std                           1.000069e+00   
min                          -1.783385e-01   
25%                          -1.783385e-01   
50%                          -1.783385e-01   
75%                          -1.783385e-01   
max                           5.607313e+00   

       remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 

,remainder__onehotencoder__Churn_Yes,remainder__onehotencoder__Churn_nan,"remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 12}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 14}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 15}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 16}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 17}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 18}",...,"remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.25, 'Total': '4652.4'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.4, 'Total': '6376.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.45, 'Total': '5957.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.5, 'Total': '4318.35'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.6, 'Total': '5817.45'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 91.25, 'Total': '6589.6'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.15, 'Total': '6480.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.45, 'Total': '6460.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 95.5, 'Total': '4627.85'}}",Churn
0,-0.588421,-0.178339,-0.059922,-0.072502,-0.067541,-0.043934,-0.063298,-0.046974,-0.043934,-0.052533,...,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,No
1,-0.588421,-0.178339,-0.059922,-0.072502,-0.067541,-0.043934,-0.063298,-0.046974,-0.043934,-0.052533,...,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,No
2,1.699463,-0.178339,-0.059922,-0.072502,-0.067541,-0.043934,-0.063298,-0.046974,-0.043934,-0.052533,...,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,Yes
3,1.699463,-0.178339,-0.059922,-0.072502,-0.067541,-0.043934,-0.063298,-0.046974,-0.043934,-0.052533,...,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,Yes
4,1.699463,-0.178339,-0.059922,-0.072502,-0.067541,-0.043934,-0.063298,-0.046974,-0.043934,-0.052533,...,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,-0.011731,Yes


In [ ]:
# Recomendações
print("\nRecomendações:")
print("- Use normalização (MinMaxScaler) se os modelos forem sensíveis a intervalos (ex.: KNN, SVM, redes neurais).")
print("- Use padronização (StandardScaler) se os modelos assumem dados com distribuição normal ou são sensíveis a outliers (ex.: regressão logística, SVM).")
print("- Se usar modelos baseados em árvores (ex.: Random Forest, XGBoost), normalização/padronização pode não ser necessária.")
print("- Verifique a distribuição dos dados (ex.: histogramas) para confirmar a escolha.")


Recomendações:
- Use normalização (MinMaxScaler) se os modelos forem sensíveis a intervalos (ex.: KNN, SVM, redes neurais).
- Use padronização (StandardScaler) se os modelos assumem dados com distribuição normal ou são sensíveis a outliers (ex.: regressão logística, SVM).
- Se usar modelos baseados em árvores (ex.: Random Forest, XGBoost), normalização/padronização pode não ser necessária.
- Verifique a distribuição dos dados (ex.: histogramas) para confirmar a escolha.


Trabalhando o Balanceamento de Classes

In [ ]:
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

In [ ]:

# Identificando colunas categóricas
colunas_categoricas = x.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
# Criando o transformador de colunas
column_transformer = make_column_transformer(
    (OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), colunas_categoricas),
    remainder='passthrough'
)

In [ ]:
# Aplicando a transformação
x_transformado = column_transformer.fit_transform(x)

In [ ]:
# Obtendo nomes das colunas após transformação
novas_colunas = column_transformer.get_feature_names_out()

In [ ]:
# Criar DataFrame com os dados transformados
x_transformado = pd.DataFrame(x_transformado, columns=novas_colunas)

In [ ]:
# Aplicando SMOTE (oversampling da classe minoritária)
smote = SMOTE(random_state=5)
# Manipulando valores na variável de destino y
y_cleaned = y.dropna()

# Certificando-se de que x_transformado corresponde ao y limpo
x_cleaned = x_transformado[y.notna()]

# Convertendo a variável de destino em numérica
y_encoded = y_cleaned.map({'No': 0, 'Yes': 1})

x_smote, y_smote = smote.fit_resample(x_cleaned, y_encoded)

In [ ]:
# Criando DataFrame com dados balanceados (SMOTE)
dados_smote = pd.concat([pd.DataFrame(x_smote, columns=novas_colunas), pd.Series(y_smote, name='Churn')], axis=1)

In [ ]:
# Exibindo a distribuição após SMOTE
print("\nDistribuição após SMOTE:")
print(y_smote.value_counts())
display(dados_smote.head())


Distribuição após SMOTE:
Churn
0    5174
1    5174
Name: count, dtype: int64


,remainder__onehotencoder__Churn_Yes,remainder__onehotencoder__Churn_nan,"remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 12}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 14}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 15}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 16}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 17}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 18}",...,"remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.25, 'Total': '4652.4'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.4, 'Total': '6376.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.45, 'Total': '5957.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.5, 'Total': '4318.35'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.6, 'Total': '5817.45'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 91.25, 'Total': '6589.6'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.15, 'Total': '6480.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.45, 'Total': '6460.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 95.5, 'Total': '4627.85'}}",Churn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
# Aplicando RandomUnderSampler (undersampling da classe majoritária)
undersampler = RandomUnderSampler(random_state=5)

# Manipulando valores na variável de destino y
y_cleaned = y.dropna()

# Verificando se x_transformado corresponde ao y limpo
x_cleaned = x_transformado[y.notna()]

# Converta a variável de destino em numérica
y_encoded = y_cleaned.map({'No': 0, 'Yes': 1})

x_under, y_under = undersampler.fit_resample(x_cleaned, y_encoded)

In [ ]:
# Criar DataFrame com dados balanceados (undersampling)
dados_under = pd.concat([pd.DataFrame(x_under, columns=novas_colunas), pd.Series(y_under, name='Churn')], axis=1)

In [ ]:
# Exibir distribuição após undersampling
print("\nDistribuição após undersampling:")
print(y_under.value_counts())
display(dados_under.head())


Distribuição após undersampling:
Churn
0    1869
1    1869
Name: count, dtype: int64


,remainder__onehotencoder__Churn_Yes,remainder__onehotencoder__Churn_nan,"remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 11}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 12}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 14}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 15}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 16}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 17}","remainder__onehotencoder__customer_{'gender': 'Female', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 18}",...,"remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.25, 'Total': '4652.4'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 89.4, 'Total': '6376.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.45, 'Total': '5957.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.5, 'Total': '4318.35'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 90.6, 'Total': '5817.45'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 91.25, 'Total': '6589.6'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.15, 'Total': '6480.9'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 92.45, 'Total': '6460.55'}}","remainder__onehotencoder__account_{'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Mailed check', 'Charges': {'Monthly': 95.5, 'Total': '4627.85'}}",Churn
6350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
# Salvando os datasets balanceados
dados_smote.to_csv('/content/dados_balanceados_smote.csv', index=False)
dados_under.to_csv('/content/dados_balanceados_undersampling.csv', index=False)

In [ ]:
# Impressão dos dados balanceados salvos
#print("\nDatasets balanceados salvos como 'dados_balanceados_smote.csv' e 'dados_balanceados_undersampling.csv'.")

🎯 Correlação e Seleção de Variáveis

In [ ]:
# Combinar features transformadas com o target
dados_completos = pd.concat([pd.DataFrame(x_transformado, columns=novas_colunas), y], axis=1)